# End-to-End Customer Churn Analysis (Clean ML Pipeline)

## Business Context
Customer churn refers to customers discontinuing a subscription-based service.
In subscription businesses, **missing a churned customer is more costly**
than incorrectly flagging a loyal one.

Therefore, this project frames churn prediction as a **cost-sensitive,
probability-based decision problem**, not just a classification task.

## Objectives
- Build a clean, reproducible churn prediction pipeline
- Prioritize recall for churned customers
- Generate churn probabilities and actionable risk segments
- Save trained artifacts for deployment


In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix

import warnings
warnings.filterwarnings("ignore")


## Data Loading
We load the Telco Customer Churn dataset and perform basic sanity checks.


In [2]:
df = pd.read_csv("../data/telco_churn.csv")

df.head()


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [3]:
df.shape


(7043, 21)

## Data Cleaning & Target Encoding
We encode the churn target and remove non-informative identifiers.


In [4]:
# Encode target variable
df["Churn"] = df["Churn"].map({"Yes": 1, "No": 0})

# Drop customer identifier
df = df.drop("customerID", axis=1)

df["Churn"].value_counts(normalize=True)


Churn
0    0.73463
1    0.26537
Name: proportion, dtype: float64

## Feature Engineering
We prepare the feature matrix using one-hot encoding.


In [6]:
X = df.drop("Churn", axis=1)
y = df["Churn"]

# One-hot encoding
X = pd.get_dummies(X, drop_first=True)

X.shape


(7043, 6559)

## Train–Test Split
We use a stratified split to preserve churn distribution.


In [7]:
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)


## Cost-Sensitive Model Training
We use class-weighted Logistic Regression to prioritize recall
for churned customers.


In [8]:
final_model = Pipeline([
    ("scaler", StandardScaler()),
    ("model", LogisticRegression(
        class_weight={0: 1, 1: 5},
        max_iter=1000
    ))
])

final_model.fit(X_train, y_train)


,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('scaler', ...), ('model', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"copy copy: bool, default=TrueIf False, try to avoid a copy and do inplace scaling instead.This is not guaranteed to always work inplace; e.g. if the data isnot a NumPy array or scipy.sparse CSR matrix, a copy may still bereturned.",True
,"with_mean with_mean: bool, default=TrueIf True, center the data before scaling.This does not work (and will raise an exception) when attempted onsparse matrices, because centering them entails building a densematrix which in common use cases is likely to be too large to fit inmemory.",True
,"with_std with_std: bool, default=TrueIf True, scale the data to unit variance (or equivalently,unit standard deviation).",True
,"penalty penalty: {'l1', 'l2', 'elasticnet', None}, default='l2'Specify the norm of the penalty:- `None`: no penalty is added;- `'l2'`: add a L2 penalty term and it is the default choice;- `'l1'`: add a L1 penalty term;- `'elasticnet'`: both L1 and L2 penalty terms are added... warning:: Some penalties may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionadded:: 0.19 l1 penalty with SAGA solver (allowing 'multinomial' + L1).. deprecated:: 1.8 `penalty` was deprecated in version 1.8 and will be removed in 1.10. Use `l1_ratio` instead. `l1_ratio=0` for `penalty='l2'`, `l1_ratio=1` for `penalty='l1'` and `l1_ratio` set to any float between 0 and 1 for `'penalty='elasticnet'`.",'deprecated'
,"C C: float, default=1.0Inverse of regularization strength; must be a positive float.Like in support vector machines, smaller values specify strongerregularization. `C=np.inf` results in unpenalized logistic regression.For a visual example on the effect of tuning the `C` parameterwith an L1 penalty, see::ref:`sphx_glr_auto_examples_linear_model_plot_logistic_path.py`.",1.0
,"l1_ratio l1_ratio: float, default=0.0The Elastic-Net mixing parameter, with `0 <= l1_ratio <= 1`. Setting`l1_ratio=1` gives a pure L1-penalty, setting `l1_ratio=0` a pure L2-penalty.Any value between 0 and 1 gives an Elastic-Net penalty of the form`l1_ratio * L1 + (1 - l1_ratio) * L2`... warning:: Certain values of `l1_ratio`, i.e. some penalties, may not wo

## Model Evaluation
We evaluate the model using recall-focused metrics.


In [9]:
y_pred = final_model.predict(X_test)

print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.85      0.82      0.83      1035
           1       0.55      0.61      0.58       374

    accuracy                           0.76      1409
   macro avg       0.70      0.71      0.70      1409
weighted avg       0.77      0.76      0.77      1409



In [10]:
confusion_matrix(y_test, y_pred)


array([[845, 190],
       [146, 228]])

### Evaluation Insight
Recall is prioritized to minimize missed churn cases,
which aligns with real-world retention objectives.


## Churn Probability & Risk Segmentation
Instead of binary decisions, we generate churn probabilities
and convert them into actionable risk tiers.


In [11]:
# Predict churn probability for all customers
df["churn_probability"] = final_model.predict_proba(X)[:, 1]

def assign_risk(prob):
    if prob >= 0.6:
        return "High Risk"
    elif prob >= 0.3:
        return "Medium Risk"
    else:
        return "Low Risk"

df["risk_segment"] = df["churn_probability"].apply(assign_risk)

df[["churn_probability", "risk_segment"]].head()


,churn_probability,risk_segment
0,6.057039e-04,Low Risk
1,1.022182e-04,Low Risk
2,9.996860e-01,High Risk
3,5.609807e-07,Low Risk
4,9.998662e-01,High Risk


## Saving Model Artifacts
We serialize the trained model and feature schema for
inference-only deployment in a Streamlit application.


In [12]:
import joblib
from pathlib import Path

# Project root (one level above notebooks/)
PROJECT_ROOT = Path.cwd().parent

# Create models directory
MODELS_PATH = PROJECT_ROOT / "models"
MODELS_PATH.mkdir(exist_ok=True)

# Save model and feature columns
joblib.dump(final_model, MODELS_PATH / "churn_model.pkl")
joblib.dump(X.columns.tolist(), MODELS_PATH / "feature_columns.pkl")

print("Saved artifacts:")
print(list(MODELS_PATH.iterdir()))


Saved artifacts:
[WindowsPath('C:/Users/Dell/Downloads/Customer-Churn-Case-Study/models/churn_model.pkl'), WindowsPath('C:/Users/Dell/Downloads/Customer-Churn-Case-Study/models/feature_columns.pkl')]
